<a href="https://colab.research.google.com/github/kelseymatsik/project_chd/blob/main/tori_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### GOAL: Build predictive algorithms that predict the likelihood a person develops coronary heart disease (CHD).

In [2]:
! git clone https://github.com/kelseymatsik/project_chd

Cloning into 'project_chd'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 37 (delta 19), reused 1 (delta 0), pack-reused 0
Receiving objects: 100% (37/37), 737.43 KiB | 3.11 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("./project_chd/fhs_train.csv")
df.head()

,Unnamed: 0,sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1267,1,58,1.0,0,0.0,0.0,0,0,0,220.0,143.0,104.0,29.85,75,87.0,1
1,1209,0,40,1.0,1,15.0,0.0,0,0,0,199.0,122.0,82.0,22.16,85,77.0,0
2,2050,0,52,1.0,0,0.0,0.0,0,0,0,275.0,112.0,71.0,25.68,80,NaN,0
3,1183,1,38,2.0,1,43.0,0.0,0,1,0,170.0,130.0,94.0,23.90,110,75.0,0
4,3225,0,43,1.0,0,0.0,0.0,0,0,0,202.0,124.0,92.0,21.26,75,74.0,0


In [5]:
# Renaming columns to be more intuitive
df = df.rename(columns = {'prevalentStroke': 'stroke',
                          'prevalentHyp': 'hypertensive',
                          'TenYearCHD': 'tenYearRisk'})

# Dropping 'Unnamed: 0' column
df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
df.head()

,sex,age,education,currentSmoker,cigsPerDay,BPMeds,stroke,hypertensive,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,tenYearRisk
0,1,58,1.0,0,0.0,0.0,0,0,0,220.0,143.0,104.0,29.85,75,87.0,1
1,0,40,1.0,1,15.0,0.0,0,0,0,199.0,122.0,82.0,22.16,85,77.0,0
2,0,52,1.0,0,0.0,0.0,0,0,0,275.0,112.0,71.0,25.68,80,NaN,0
3,1,38,2.0,1,43.0,0.0,0,1,0,170.0,130.0,94.0,23.90,110,75.0,0
4,0,43,1.0,0,0.0,0.0,0,0,0,202.0,124.0,92.0,21.26,75,74.0,0


In [7]:
df.columns

Index(['sex', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds',
       'stroke', 'hypertensive', 'diabetes', 'totChol', 'sysBP', 'diaBP',
       'BMI', 'heartRate', 'glucose', 'tenYearRisk'],
      dtype='object')

In [8]:
# Setting temporary variable names
sex = df['sex']
age = df['age']
education = df['education']
currentSmoker = df['currentSmoker']
cigsPerDay = df['cigsPerDay']
BPMeds = df['BPMeds']
stroke = df['stroke']
hypertensive = df['hypertensive']
diabetes = df['diabetes']
totChol = df['totChol']
sysBP = df['sysBP']
diaBP = df['diaBP']
BMI = df['BMI']
heartRate = df['heartRate']
glucose = df['glucose']
glucose = df['glucose']

In [9]:
# Looking at value counts for each variable

#for var in df.columns:
#  print(df[var].value_counts())

# cigsPerDay is the number of cigarettes smoked each day, but only has values of 0 and 1
# Not sure what that means...

In [10]:
for var in df.columns:
  print(df[var].isna().sum()/len(df))

0.0
0.0
0.026729559748427674
0.0
0.007547169811320755
0.011635220125786163
0.0
0.0
0.0
0.012264150943396227
0.0
0.0
0.0047169811320754715
0.0
0.08962264150943396
0.0


In [11]:
# Check for missings
for var in df.columns:
    missing_percentage = df[var].isna().sum() / len(df) * 100
    print(f"{var}: {missing_percentage:.2f}% missing")
# Overall, the data frame does not have many missings

sex: 0.00% missing
age: 0.00% missing
education: 2.67% missing
currentSmoker: 0.00% missing
cigsPerDay: 0.75% missing
BPMeds: 1.16% missing
stroke: 0.00% missing
hypertensive: 0.00% missing
diabetes: 0.00% missing
totChol: 1.23% missing
sysBP: 0.00% missing
diaBP: 0.00% missing
BMI: 0.47% missing
heartRate: 0.00% missing
glucose: 8.96% missing
tenYearRisk: 0.00% missing


In [12]:
# Imputation for education, cigsPerDay, BPMeds, totChol, BMI, glucose
df['education'].value_counts()
# Only 1-4, so don't impute with averages here ?
# OK b/c not much missing

1.0    1310
2.0     949
3.0     495
4.0     341
Name: education, dtype: int64

In [13]:
df['cigsPerDay'].value_counts().head(5)

0.0     1619
20.0     528
30.0     172
15.0     159
10.0     108
Name: cigsPerDay, dtype: int64

In [14]:
df['BPMeds'].value_counts()
# Only 1 or 0, so don't impute with averages here ?
# OK b/c not much missing

0.0    3050
1.0      93
Name: BPMeds, dtype: int64

In [15]:
df['totChol'].value_counts().head(5)

240.0    63
220.0    50
232.0    49
260.0    48
210.0    45
Name: totChol, dtype: int64

In [16]:
df['BMI'].value_counts().head(5)

23.48    17
22.91    15
22.54    12
23.09    12
25.09    11
Name: BMI, dtype: int64

In [17]:
df['glucose'].value_counts().head(5)

75.0    147
77.0    126
73.0    119
80.0    117
70.0    113
Name: glucose, dtype: int64

In [18]:
# Check for NaN overlaps
both_nan = df[(df['education'].isna()) & (df['cigsPerDay'].isna())]
education_nan_cigsPerDay_not = df[(df['education'].isna()) & (df['cigsPerDay'].notna())]
cigsPerDay_nan_education_not = df[(df['cigsPerDay'].isna()) & (df['education'].notna())]

# Output the counts
print(f"Rows where both variables are NaN: {len(both_nan)}")
print(f"Rows where 'education' is NaN and 'cigsPerDay' is not: {len(education_nan_cigsPerDay_not)}")
print(f"Rows where 'cigsPerDay' is NaN and 'education' is not: {len(cigsPerDay_nan_education_not)}")

Rows where both variables are NaN: 2
Rows where 'education' is NaN and 'cigsPerDay' is not: 83
Rows where 'cigsPerDay' is NaN and 'education' is not: 22


In [19]:
# Check for NaN overlaps
both_nan = df[(df['education'].isna()) & (df['totChol'].isna())]
education_nan_totChol_not = df[(df['education'].isna()) & (df['totChol'].notna())]
totChol_nan_education_not = df[(df['totChol'].isna()) & (df['education'].notna())]

# Output the counts
print(f"Rows where both variables are NaN: {len(both_nan)}")
print(f"Rows where 'education' is NaN and 'totChol' is not: {len(education_nan_totChol_not)}")
print(f"Rows where 'totChol' is NaN and 'education' is not: {len(totChol_nan_education_not)}")

Rows where both variables are NaN: 1
Rows where 'education' is NaN and 'totChol' is not: 84
Rows where 'totChol' is NaN and 'education' is not: 38


In [20]:
# Check for NaN overlaps
both_nan = df[(df['education'].isna()) & (df['BMI'].isna())]
education_nan_BMI_not = df[(df['education'].isna()) & (df['BMI'].notna())]
BMI_nan_education_not = df[(df['BMI'].isna()) & (df['education'].notna())]

# Output the counts
print(f"Rows where both variables are NaN: {len(both_nan)}")
print(f"Rows where 'education' is NaN and 'BMI' is not: {len(education_nan_BMI_not)}")
print(f"Rows where 'BMI' is NaN and 'education' is not: {len(BMI_nan_education_not)}")

Rows where both variables are NaN: 1
Rows where 'education' is NaN and 'BMI' is not: 84
Rows where 'BMI' is NaN and 'education' is not: 14


In [21]:
# Check for NaN overlaps
both_nan = df[(df['education'].isna()) & (df['glucose'].isna())]
education_nan_glucose_not = df[(df['education'].isna()) & (df['glucose'].notna())]
glucose_nan_education_not = df[(df['glucose'].isna()) & (df['education'].notna())]

# Output the counts
print(f"Rows where both variables are NaN: {len(both_nan)}")
print(f"Rows where 'education' is NaN and 'glucose' is not: {len(education_nan_glucose_not)}")
print(f"Rows where 'glucose' is NaN and 'education' is not: {len(glucose_nan_education_not)}")

Rows where both variables are NaN: 6
Rows where 'education' is NaN and 'glucose' is not: 79
Rows where 'glucose' is NaN and 'education' is not: 279


In [22]:
# Check for rows where both 'education' and 'BPMeds' are NaN
both_nan = df[(df['education'].isna()) & (df['BPMeds'].isna())]

# Output the count
print(f"Rows where both 'education' and 'BPMeds' are NaN: {len(both_nan)}")

Rows where both 'education' and 'BPMeds' are NaN: 0


Since there are no rows where both education and BPMeds are NA, we will end up dropping 3.83% of the data and imputing values for the rest of the columns with missing values, which isn't too bad.

In [23]:
# Data types of missings
print('education: ', education.dtype)
print('cigsPerDay: ', cigsPerDay.dtype)
print('BPMeds: ', BPMeds.dtype)
print('totChol: ', totChol.dtype)
print('BMI: ', BMI.dtype)
print('glucose: ', glucose.dtype)

education:  float64
cigsPerDay:  float64
BPMeds:  float64
totChol:  float64
BMI:  float64
glucose:  float64
